### Partice Swarm Optimization 

In [1]:
import osmnx as ox
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Define the objective function to minimize (distance between nodes)
def objective_function(route, graph):
    distance = 0
    for i in range(len(route) - 1):
        distance += ox.distance.euclidean_dist_vec(graph.nodes[route[i]]['y'], 
                                                    graph.nodes[route[i]]['x'], 
                                                    graph.nodes[route[i+1]]['y'], 
                                                    graph.nodes[route[i+1]]['x'])
    return distance

In [3]:
# Defining the Particle class
class Particle:
    def __init__(self, graph, num_nodes):
        self.position = np.random.permutation(range(num_nodes))
        self.velocity = np.zeros(num_nodes)
        self.best_position = self.position.copy()
        self.best_score = objective_function(self.position, graph)

    def update_velocity(self, global_best_position, omega, phi_p, phi_g):
        r_p, r_g = np.random.random(2)
        self.velocity = omega * self.velocity + phi_p * r_p * (self.best_position - self.position) + phi_g * r_g * (global_best_position - self.position)

    def update_position(self, graph):
        self.position = np.roll(self.position, np.random.randint(len(self.position)))
        score = objective_function(self.position, graph)
        if score < self.best_score:
            self.best_position = self.position.copy()
            self.best_score = score

In [4]:
# Defining the PSO function
def pso(graph, num_particles, max_iter):
    # Initialize the particles
    particles = [Particle(graph, len(graph.nodes)) for _ in range(num_particles)]

    # Initialize the global best position
    global_best_position = particles[0].best_position.copy()
    global_best_score = particles[0].best_score

    # Run the PSO algorithm
    for i in range(max_iter):
        for particle in particles:
            particle.update_velocity(global_best_position, 0.5, 0.5, 0.5)
            particle.update_position(graph)
            if particle.best_score < global_best_score:
                global_best_position = particle.best_position.copy()
                global_best_score = particle.best_score

    # Returning the best path
    return global_best_position

In [5]:
# Defining the main function to plot the path
def plot_path(place):
    # Downloading the street network
    graph = ox.graph_from_place(place, network_type='drive')

    # Running the PSO algorithm to find the best path
    path = pso(graph, num_particles=20, max_iter=50)

    # Plotting the path on the street network
    fig, ax = ox.plot_graph_route(graph, path, fig_height=10, node_size=0, edge_color='r', edge_linewidth=2)

    # Showing the plot
    plt.show()

In [8]:
#the main function
place = 'Koramangala, Bangalore, India'
plot_path(place)


KeyboardInterrupt

